In [1]:
%load_ext autoreload
%autoreload 2


In [ ]:
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import pytz

from models.gan_models import *
import yfinance as yf
import MetaTrader5 as mt5

In [ ]:
experiment = Experiment(
  api_key="vpNJF6XOWcHS6HqH9ZFEjwRcD",
  project_name="tail-price",
  workspace="artaasd95"
)

In [29]:
mt5.initialize()

print('loading current tf data')
utc_from = datetime(2022, 1, 1, tzinfo=pytz.timezone("Asia/Nicosia"))
utc_to = datetime.now(pytz.timezone("Asia/Nicosia"))

data = mt5.copy_rates_range('XAUUSD', mt5.TIMEFRAME_D1, utc_from, utc_to)
data = pd.DataFrame(data)
time_data = data.time
data.drop(columns=['tick_volume', 'spread', 'real_volume'], inplace=True)

loading current tf data


In [30]:
data

,time,open,high,low,close
0,1641168000,1828.42,1831.58,1798.18,1801.10
1,1641254400,1800.42,1816.58,1798.38,1814.27
2,1641340800,1813.82,1829.49,1808.15,1810.54
3,1641427200,1808.58,1812.07,1786.22,1789.52
4,1641513600,1790.42,1798.52,1782.41,1796.12
...,...,...,...,...,...
673,1723161600,2425.94,2437.02,2416.80,2430.82
674,1723420800,2428.05,2472.17,2423.83,2472.06
675,1723507200,2470.68,2476.97,2458.46,2465.41
676,1723593600,2466.51,2479.94,2438.05,2447.66


In [31]:
close_price = data.close

In [ ]:
input_size = 1
hidden_size = 512
seq_length = 50
num_layers = 5
batch_size = 256
num_epochs = 100
learning_rate = 0.2

# Initialize models
main_gen = MainGenerator(input_size, hidden_size, num_layers, 0.5, 1)
noise_gen = NoiseGenerator(input_size, hidden_size, 1, 0.3)
discriminator = Discriminator(input_size, hidden_size, 10, 0.3)

In [ ]:
optimizer_G = torch.optim.AdamW(list(main_gen.parameters()) + list(noise_gen.parameters()), lr=learning_rate)
optimizer_D = torch.optim.AdamW(discriminator.parameters(), lr=learning_rate)

optim_g_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_G, 0.1)
optim_d_sched = torch.optim.lr_scheduler.ExponentialLR(optimizer_D, 0.1)

In [45]:
cauchy_dist = torch.distributions.cauchy.Cauchy(loc=0, scale=0.5)

In [51]:
cauchy_dist.sample([1, 1])

tensor([[-0.2385]])

In [ ]:
# Training loop
for epoch in tqdm(range(num_epochs)):
    for idx in len(close_price):
        price = torch.from_numpy(data.iloc[idx].close)
        # Generate random noise inputs
        z1 = cauchy_dist.sample([1])
        #z2 = cauchy_dist.sample([1])
        
        # Generate fake data
        fake_main = main_gen(price)
        fake_noise = noise_gen(z1)
        fake_data = fake_main + fake_noise
        
        # Train discriminator
        optimizer_D.zero_grad()
        real_output = discriminator(price)
        fake_output = discriminator(fake_data.detach())
        d_loss = discriminator_loss(real_output, fake_output)
        d_loss.backward()
        optimizer_D.step()
        
        # Train generators
        optimizer_G.zero_grad()
        fake_output = discriminator(fake_data)
        g_loss = generator_loss(fake_output)
        g_loss.backward()
        optimizer_G.step()
    
    optim_g_sched.step()
    optim_d_sched.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}]  Discriminator Loss: {d_loss.item():.4f}  Generator Loss: {g_loss.item():.4f}")
